In [43]:
import pandas as pd
data = pd.read_excel("Online retail.xlsx", sheet_name="Sheet1", header=None)
#header is none beacause there are no column names in our dataset
#if header is not none pandas consider the first row as header

# Data Preprocessing:

In [19]:
data.head()

,0
0,"shrimp,almonds,avocado,vegetables mix,green gr..."
1,"burgers,meatballs,eggs"
2,chutney
3,"turkey,avocado"
4,"mineral water,milk,energy bar,whole wheat rice..."


In [20]:
data.shape

(7501, 1)

In [21]:
#in our dataset, each row is representing a transaction
#each item bought in that transaction is given as a comma seperated value
#this is not a regular tabular format data so we need to convert it to the tabular form

In [22]:
#Removing missing values and duplicates
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)

#Splitinng each transaction string into list of items as required for encoding them
transactions = data.iloc[:, 0].apply(lambda x: [item.strip().lower() for item in x.split(',') if item.strip() != ''])
transactions.head()

0    [shrimp, almonds, avocado, vegetables mix, gre...
1                           [burgers, meatballs, eggs]
2                                            [chutney]
3                                    [turkey, avocado]
4    [mineral water, milk, energy bar, whole wheat ...
Name: 0, dtype: object

In [27]:
#one hot encoding the categorical data
from mlxtend.preprocessing import TransactionEncoder

#converting transaction list to one-hot encoded dataframe 
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
data_encoded = pd.DataFrame(te_array, columns=te.columns_).astype(int)

data_encoded.head()


,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,body spray,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,1,1,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [35]:
#Now our dataset is ready for Association Rule Mining:

# Association Rule Mining:


In [39]:
from mlxtend.frequent_patterns import apriori, association_rules

#Applying Apriori to find frequent itemsets
frequent_itemsets = apriori(data_encoded, min_support=0.02, use_colnames=True)

#Generating association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

#Filtering rules with lift > 1 for positive associations
rules = rules[rules['lift'] > 1.0]

#Sorting rules by lift 
rules = rules.sort_values(by="lift", ascending=False)

#Shows top 10 rules
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)


/Users/nikhil/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,support,confidence,lift
14,(herb & pepper),(ground beef),0.022798,0.343023,2.525588
39,"(ground beef, mineral water)",(spaghetti),0.024730,0.421053,1.834485
16,(ground beef),(spaghetti),0.055835,0.411095,1.791102
22,(soup),(milk),0.021445,0.302452,1.778970
32,(olive oil),(spaghetti),0.032457,0.370044,1.612246
27,(soup),(mineral water),0.033423,0.471390,1.572109
0,(burgers),(eggs),0.036128,0.317487,1.525826
18,(herb & pepper),(spaghetti),0.023184,0.348837,1.519850
42,"(milk, spaghetti)",(mineral water),0.022604,0.451737,1.506568
35,"(milk, chocolate)",(mineral water),0.020093,0.448276,1.495023


# Analysis and Interpretation:

In [40]:
#herb & pepper , ground beef Support: 2.3%,Confidence: 34%,Lift: 2.52
#Customers who buy herb & pepper are 2.5× more likely to buy ground beef 

#ground beef & mineral water,spaghetti ,ground beef & spaghetti,olive oil & paghetti
#customers buy Spaghetti frequently with ground beef, olive oil, and spices.

#soup & milk,soup & mineral water
#People buying soup are very likely to buy milk or mineral water as well

# Interview Questions:


In [41]:
#What is lift and why is it important in Association rules?
#Lift measures how much more likely two items are to be bought together than if they were independent.
# Lift > 1: Positive correlation
# Lift = 1: No correlation
# Lift < 1: Negative correlation

In [42]:
#What is support and Confidence. How do you calculate them?
#support gives the prportion of transactions when given a particular item
#Support= transactions containing a intersection b/total transactions

#confidence gives the liklihodd of a item that will be purchased when other item is give
#confidence=supoort of a intersection b/support of a
